In [9]:
#!wget https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.7.0/ru_core_news_sm-3.7.0-py3-none-any.whl -o ru_core_news_sm

In [8]:
import spacy

nlp = spacy.load("ru_core_news_sm")

text = "Способ спасти посевы после ледяного дождя он назвал в беседе с aif.ru. По его словам, главный вред, который может причинить ледяной дождь посеянным осенью чесноку, моркови, свекле и другим посадкам — помешать воздухообмену. Если дождь прошел после того, как дачник убрал снег над посадками или если на участке мало снега, то надо побыстрее ликвидировать ледяную корку, образовавшуюся на поверхности, уточнил он. «Делать это надо грубым механическим путем, разбив ее граблями — лучше теми, которыми вы убираете листву на участке. Избавившись от корки и убрав образовавшуюся мешанину подальше, восстановите снежный покров, хорошенько встряхивая снег, чтобы он был сыпучим, а не слежавшимся», — порекомендовал он. Проливать огород горячей водой, продолжил Синьковский, нельзя. Если же ледяной дождь прошел над грядками, находящимися под нормальной толщины снежным покровом, нужно по всей площади грядок сломать образовавшийся наст, посоветовал он. Этот наст, объяснил агроном, нужно разбить на фрагменты и убрать с поверхности, а потом черенком от лопаты пробить отверстия по всей площади грядок. «Ничего, что они быстро заполнятся снегом — должную циркуляцию воздуха вы таким образом все равно обеспечите», — заверил специалист. Ранее Синьковский назвал правила выращивания баклажанов и перцев. Агроном рекомендовал использовать готовые жирные грунты с высоким содержанием органики."

doc = nlp(text)

# Извлекаем ключевые слова на основе частоты
keywords_freq = [token.text for token in doc if not token.is_stop and token.is_alpha]

# Извлекаем ключевые слова на основе веса встроенных векторов
keywords_semantic = [token.text for token in doc if not token.is_stop and token.vector_norm > 0]

print("Keywords based on frequency:", keywords_freq)
print("Keywords based on semantics:", keywords_semantic)

Keywords based on frequency: ['Способ', 'спасти', 'посевы', 'ледяного', 'дождя', 'назвал', 'беседе', 'словам', 'главный', 'вред', 'причинить', 'ледяной', 'дождь', 'посеянным', 'осенью', 'чесноку', 'моркови', 'свекле', 'посадкам', 'помешать', 'воздухообмену', 'дождь', 'прошел', 'дачник', 'убрал', 'снег', 'посадками', 'участке', 'снега', 'побыстрее', 'ликвидировать', 'ледяную', 'корку', 'образовавшуюся', 'поверхности', 'уточнил', 'Делать', 'грубым', 'механическим', 'путем', 'разбив', 'граблями', 'лучше', 'убираете', 'листву', 'участке', 'Избавившись', 'корки', 'убрав', 'образовавшуюся', 'мешанину', 'подальше', 'восстановите', 'снежный', 'покров', 'хорошенько', 'встряхивая', 'снег', 'сыпучим', 'слежавшимся', 'порекомендовал', 'Проливать', 'огород', 'горячей', 'водой', 'продолжил', 'Синьковский', 'ледяной', 'дождь', 'прошел', 'грядками', 'находящимися', 'нормальной', 'толщины', 'снежным', 'покровом', 'площади', 'грядок', 'сломать', 'образовавшийся', 'наст', 'посоветовал', 'наст', 'объяснил

In [10]:
!pip install requests
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [14]:
import requests as req 
from bs4 import BeautifulSoup
import pandas as pd
import time
from collections import deque


class RiaParser:
    '''Культура'''
    routes = [
        {
            'category': 'Политика',
            'keyword': 'politics'
        },
        {
            'category': 'Экономика',
            'keyword': 'economy'
        },
        {
            'category': 'Общество',
            'keyword': 'society'
        },
        {
            'category': 'Кино',
            'keyword': 'organization_Gruppa_Kino'
        },
        {
            'category': 'Законы',
            'keyword': 'tag_zakon'
        },
        {
            'category': 'Телевидение',
            'keyword': 'category_televidenie'
        },
        {
            'category': 'Люди',
            'keyword': 'keyword_ljudi'
        },
        {
            'category': 'Бренды',
            'keyword': 'keyword_brendy'
        },
        {
            'category': 'Наука',
            'keyword': 'tag_gadzhety'
        },
        {
            'category': 'Гаджеты',
            'keyword': 'tag_gadzhety'
        },
        {
            'category': 'Соцсети',
            'keyword': 'tag_Socseti'
        },
        {
            'category': 'Технологии',
            'keyword': 'technology'
        },
        {
            'category': 'Опросы',
            'keyword': 'polls'
        },
        {
            'category': 'Транспорт',
            'keyword': 'tag_thematic_category_Transport'
        },
        {
            'category': 'Погода',
            'keyword': 'category_pogoda'
        },
        {
            'category': 'Рецепты',
            'keyword': 'category_retsepty'
        },
        {
            'category': 'Мода',
            'keyword': 'tag_moda_2'
        },
        {
            'category': 'Красота',
            'keyword': 'product_krasota'
        },
    ]

    df = pd.DataFrame({'category': [], 'text': []})
    f = open('out_parsed_data.csv', 'a', encoding='utf-8')
    f.write('category')
    f.write(';')
    f.write('data')
    f.write('\n')
    deq = deque()

    @classmethod
    def get_ref_refs(cls, n):
        ret = []
        
        return ret
    
    @classmethod
    def get_refs_page(cls, start_url, n):
        pass


    @classmethod
    def get_refs_news(cls, soup):
        ret = []
        # soup = BeautifulSoup(html, 'html.parser')
        elements = soup.find_all('a', attrs={'class': 'list-item__title color-font-hover-only'})
        for element in elements:
            ret.append(element['href'])
        return ret
    
    @classmethod
    def get_text(cls, url):
        if 1:
            res = req.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')

            topics = soup.find_all('div', attrs={'class': 'article__block'})
            topics_text = []
            for topic in topics:
                topics_text.append(topic.text)
            return ' '.join(topics_text)
        return ''

    @classmethod
    def save_to_df(cls, category, data):
        cls.df.loc[len(cls.df)] = [category, data]

    @classmethod
    def save_to_file(cls, category, data):
        cls.f.write(category)
        cls.f.write(';')
        cls.f.write(data)
        cls.f.write('\n')
    
    @classmethod
    def save_deque_to_file(cls):
        cls.deq
        while cls.deq:
            cat, text = cls.deq.pop()
            cls.f.write(cat)
            cls.f.write(';')
            cls.f.write(text)
            cls.f.write('\n')
        cls.f.flush()


    @classmethod
    def get_dataset(cls, start_cat=0, end_cat=len(routes)):
        for route in cls.routes[start_cat:end_cat]:
            category = route['category']
            keyword = route['keyword']
            url = 'https://ria.ru/' + keyword
            res = req.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')
            for ref in cls.get_refs_news(soup):
                # cls.save_to_file(category, cls.get_text(ref))
                cls.deq.append((category, cls.get_text(ref)))
                # cls.save_to_df(category, cls.get_text(ref))
            # cls.f.flush()
            
            url = 'https://ria.ru' + soup.find('div', attrs={'class': 'list-more color-btn-second-hover'})['data-url']
            res = req.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')
            for ref in cls.get_refs_news(soup):
                # cls.save_to_df(category, cls.get_text(ref))
                cls.deq.append((category, cls.get_text(ref)))
            

            for i in range(4):
                try:
                    url = 'https://ria.ru' + soup.find('div', attrs={'class': 'list-items-loaded'})['data-next-url']
                    res = req.get(url)
                    soup = BeautifulSoup(res.text, 'html.parser')
                    for ref in cls.get_refs_news(soup):
                        # cls.save_to_df(category, cls.get_text(ref))
                        cls.deq.append((category, cls.get_text(ref)))
                except:
                    pass

            
            # with open('out.html', 'w', encoding='utf-8') as f:
            #     f.write(res.text)
            cls.save_deque_to_file()
            print('end', category)

start = time.time()
deq = RiaParser.get_dataset()
end = time.time()
print(end - start)

end Политика
end Экономика
end Общество
end Кино
end Законы
end Телевидение
end Люди
end Бренды
end Наука
end Гаджеты
end Соцсети
end Технологии
end Опросы
end Транспорт
end Погода
end Рецепты
end Мода
end Красота
1659.3625235557556


In [16]:
import pandas as pd
df_clf = pd.read_csv("out_parsed_data.csv")

ParserError: Error tokenizing data. C error: Expected 8 fields in line 3, saw 12
